##

In [15]:
from matplotlib import pyplot as plt
import numpy as np
import torch

import torchvision
from torchvision import datasets, transforms

from collections import Counter
from torch.utils.data import random_split

In [16]:
seed = 10
torch.manual_seed(seed)

category_index_manual = 8
n_val_manual = 5000

data_path_manual = '/cifar-10-batches-py'

Function for loading the Cifar10 dataset.

The method will have to be run twice. The first time we load it as tensors and split the set between training, validation, and testing. As this is a binary classification problem where we only want to identify whether an image is a ship or not, we can set all labels that are not "ship" to false.

After running the method for the first time we get create a normalizer from the std and mean of the images. The method is then ran for a second time with the normalizer as the preprocessor.

In [17]:
def load_dataset(n_val, category_index, data_path, preprocessor):
    transformed_cifar10_train_val = datasets.CIFAR10(
        data_path,
        train=True,
        download=False,
        transform = transforms.ToTensor()
    )

    transformed_cifar10_test = datasets.CIFAR10(
        data_path,
        train=False,
        download=False,
        transform = transforms.ToTensor()
    )

    train_labels = np.array(transformed_cifar10_train_val)
    train_labels = np.array(train_labels == category_index).astype(int)

    test_labels = np.array(transformed_cifar10_test)
    test_labels = np.array(test_labels == category_index).astype(int)

    n_train = len(transformed_cifar10_train_val)-n_val

    transformed_cifar10_train_split, transformed_cifar10_val_split = random_split(
        transformed_cifar10_train_val,
        [n_train, n_val],

        generator=torch.Generator().manual_seed(seed)
    )

    print("Size of the train dataset:        ", len(transformed_cifar10_train_split))
    print("Size of the validation dataset:   ", len(transformed_cifar10_val_split))
    print("Size of the test dataset:         ", len(transformed_cifar10_test))

    return transformed_cifar10_train_split, transformed_cifar10_val_split, transformed_cifar10_test, train_labels, test_labels

cifar10_train, cifar10_val, cifar10_test, train_labels, test_labels = load_dataset(n_val_manual, category_index_manual, data_path_manual, preprocessor=transforms.ToTensor())

C:\Users\osten\AppData\Local\Temp\ipykernel_2880\1344916318.py:16: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  train_labels = np.array(transformed_cifar10_train_val)
C:\Users\osten\AppData\Local\Temp\ipykernel_2880\1344916318.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_labels = np.array(transformed_cifar10_train_val)
C:\Users\osten\AppData\Local\Tem

Size of the train dataset:         45000
Size of the validation dataset:    5000
Size of the test dataset:          10000


Creating a normalizer for the dataset around the mean and standard deviation of the 3 dimensions (height, width channel (color)).

Re-loading the dataset and applying the composition of transforms.

In [23]:
imgs = torch.stack([img for img, _ in cifar10_train])

normalizer = transforms.Normalize(mean = imgs.mean(dim=(0, 2, 3)), std = imgs.std(dim=(0, 2, 3)))

cifar10_train, cifar10_val, cifar10_test, train_labels, test_labels = load_dataset(n_val_manual, category_index_manual, data_path_manual, preprocessor=transforms.Compose([transforms.ToTensor(),normalizer]))

C:\Users\osten\AppData\Local\Temp\ipykernel_2880\1344916318.py:16: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  train_labels = np.array(transformed_cifar10_train_val)
C:\Users\osten\AppData\Local\Temp\ipykernel_2880\1344916318.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  train_labels = np.array(transformed_cifar10_train_val)
C:\Users\osten\AppData\Local\Tem

Size of the train dataset:         45000
Size of the validation dataset:    5000
Size of the test dataset:          10000


C:\Users\osten\AppData\Local\Temp\ipykernel_2880\1344916318.py:19: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  test_labels = np.array(transformed_cifar10_test)
C:\Users\osten\AppData\Local\Temp\ipykernel_2880\1344916318.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_labels = np.array(transformed_cifar10_test)
C:\Users\osten\AppData\Local\Temp\ipykernel_

Choosing a pre-trained CNN model. We choose ResNet18, which is not trained on Cifar-10.

In [19]:
from torchvision import models
import torch.nn as nn
import torch.optim as optim

In [20]:
# Loading pre-trained ResNet18 model
model = models.resnet18(pretrained=True)

# Modify the last layer. We are doing binary classification, so i think we only need one node in the final layer.
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,1)

# "Remember to use a suitable loss function like Binary Cross Entropy with Logits Loss (BCEWithLogitsLoss) and an optimizer like Adam or SGD for training.
loss_function = nn.BCEWithLogitsLoss()

# Legge til spørsmål om vi burde bruke SGD, Adam, RMSprop eller noe annet
# Legge til spørsmål om vi burde endre på learning raten, eller i alle fall forklare hvorfor vi bruker den vi bruker
optimizer = optim.adam(model.parameters(),  lr=0.001)

C:\Users\osten\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\osten\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\osten/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

AttributeError: module 'torch.optim' has no attribute 'adam'

In [ ]:
Counter([label for _, label in cifar10_train])

In [ ]:
imgs = torch.stack([img for img, _ in tensor_cifar10_train_val])
print(imgs.shape)

Creating a normalizer for the dataset around the mean and standard deviation of the 3 dimensions (height, width channel (color)).

In [ ]:
normalizer = transforms.Normalize(mean = imgs.mean(dim=(0, 2, 3)), std = imgs.std(dim=(0, 2, 3)))

Loading the dataset and applying the composition of transforms.

Attempt at plotting the normalized images.

In [ ]:
img_t = tensor_cifar10_train_val.data[1]
print(img_t)
print("halla")
print(img_t.data)

In [ ]:
print(tensor_cifar10_train_val)

In [ ]:
plt.imshow(img_t.permute(1, 2, 0))
plt.show()